In [9]:
pip install pandas azure-storage-blob

Note: you may need to restart the kernel to use updated packages.


In [12]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO

# Azure Storage account information
account_name = 'cis4400pp'
account_key = 'v5P+piRvmQQjp+PYMyBTR6oV8JSjJIYxUZCpEnDrTdsKqduDPFGOAhN/vLo2r8H3ye873+A2IG7K+AStmNDHsQ=='
container_name = 'combineddata'
blob_name = 'combined_data.csv'
output_blob_name = 'updated_combined_data.csv'

# Connect to Azure Storage Blob
blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)
container_client = blob_service_client.get_container_client(container_name)
blob_client = container_client.get_blob_client(blob_name)

# Download CSV file from Azure Blob Storage
blob_data = blob_client.download_blob()
df = pd.read_csv(StringIO(blob_data.readall().decode('utf-8')))

# Drop redundant columns
redundant_columns = ['MSA','PPM_FLAG','AMORTIZATION_TYPE','LOAN_PURPOSE','SUPER_CONFORMING_FLAG',
                     'PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER','PROGRAM_INDICATOR','RELIEF_REFINANCE_INDICATOR',
                     'PROPERTY_VALUATION_METHOD','INTEREST_ONLY_INDICATOR','MI_CANCELLATION_INDICATOR']

df = df.drop(columns = redundant_columns, errors='ignore')

# Format columns to YYYY-MM
date_columns = ['FIRST_PAYMENT_DATE','MATURITY_DATE']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], format='%Y%m', errors='coerce').dt.strftime('%Y-%m')

# Convert columns to numeric
numeric_columns = ['CREDIT_SCORE', 'POSTAL_CODE','ORIGINAL_UPB','ORIGINAL_LOAN_TERM','NUMBER_OF_UNITS','NUMBER_OF_BORROWERS']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Convert columns to percentage
percentage_columns = ['MORTGAGE_INSURANCE_PERCENTAGE','ORIGINAL_CLTV','ORIGINAL_DTI_RATIO','ORIGINAL_LTV','ORIGINAL_INTEREST_RATE']
df[percentage_columns] = df[percentage_columns].apply(pd.to_numeric, errors='coerce')/100

print(df.head())

# Save the DataFrame to a local CSV file
csv_filename = 'updated_combined_data.csv'
df.to_csv(csv_filename, index=False)

# Connect to Azure Storage Blob for the output
output_blob_client = container_client.get_blob_client(output_blob_name)

# Upload the updated CSV file to the "combineddata" container
with open(csv_filename, 'rb') as data:
    output_blob_client.upload_blob(data, blob_type="BlockBlob", content_settings=ContentSettings(content_type='csv'))

print(f"Updated data uploaded to 'combineddata' container as {output_blob_name}")

   CREDIT_SCORE FIRST_PAYMENT_DATE FIRST_TIME_HOMEBUYER_FLAG MATURITY_DATE  \
0           714            2020-03                         Y       2050-02   
1           767            2020-03                         N       2035-02   
2           770            2020-03                         N       2050-02   
3           803            2020-03                         N       2050-02   
4           717            2020-03                         N       2050-02   

   MORTGAGE_INSURANCE_PERCENTAGE  NUMBER_OF_UNITS OCCUPANCY_STATUS  \
0                            0.0                1                P   
1                            0.0                1                S   
2                            0.3                1                P   
3                            0.0                1                I   
4                            0.0                1                P   

   ORIGINAL_CLTV  ORIGINAL_DTI_RATIO  ORIGINAL_UPB  ...  \
0           0.60                0.31        400000 